Reetham & Jaan debug Mount Sinai import https://data.payless.health/#hospital_price_transparency/330024 following https://nbviewer.org/github/onefact/datathinking.org-codespace/blob/main/notebooks/princeton-university/week-1-visualizing-33-million-phone-calls-in-new-york-city.ipynb


In [2]:
# Load duckdb, which lets us efficiently load large files
import duckdb

# Load pandas, which lets us manipulate dataframes
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Set configrations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True

%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Connect jupysql to DuckDB using a SQLAlchemy-style connection string. Either connect to an in memory DuckDB, or a file backed db.
%sql duckdb:///:memory:

In [3]:
!wget https://data.payless.health/hospital_price_transparency%2F330024_131624096_mount-sinai-hospital_standardcharges.csv

--2023-08-09 06:54:09--  https://data.payless.health/hospital_price_transparency%2F330024_131624096_mount-sinai-hospital_standardcharges.csv
Resolving data.payless.health (data.payless.health)... 2600:9000:2511:4a00:1f:282d:1900:93a1, 2600:9000:2511:2800:1f:282d:1900:93a1, 2600:9000:2511:b000:1f:282d:1900:93a1, ...
Connecting to data.payless.health (data.payless.health)|2600:9000:2511:4a00:1f:282d:1900:93a1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7386687 (7.0M) [text/csv]
Saving to: ‘hospital_price_transparency%2F330024_131624096_mount-sinai-hospital_standardcharges.csv’

hospital_price_tran 100%[===================>]   7.04M  6.15MB/s    in 1.1s    

2023-08-09 06:54:11 (6.15 MB/s) - ‘hospital_price_transparency%2F330024_131624096_mount-sinai-hospital_standardcharges.csv’ saved [7386687/7386687]



In [4]:
!head hospital_price_transparency%2F330024_131624096_mount-sinai-hospital_standardcharges.csv

Date current as of 9/27/21,,,,,,,,
,,,,,,,,
CAMPUS,Payer/Plan Name,DRG/CPT/HCPCS Code,DRG/CPT/HCPCS Description,Gross Charge,Discounted Cash Price,Negotiated Charge,Minimum Negotiated Charge,Maximum Negotiated Charge
MSH,AETNA AETNA CHOICE  POS II NET 00357,1,Heart transplant or implant of heart assist system with MCC,Not Applicable,100% of Medicare,56984,37063,898459
MSH,NYSDOH,1,Heart transplant or implant of heart assist system with MCC,Not Applicable,100% of Medicare,37063,37063,898459
MSH,UNITEDHEALTHCARE OF NEW YORK INC NY DUAL COMPLETE,1,Heart transplant or implant of heart assist system with MCC,Not Applicable,100% of Medicare,250698,37063,898459
MSH,AETNA ADV PLAN 250 NET 03397,1,Heart transplant or implant of heart assist system with MCC,Not Applicable,100% of Medicare,222752,37063,898459
MSH,OXFORD HEALTH INSURANCE INC FREEDOM,1,Heart transplant or implant of heart assist system with MCC,Not Applicable,100% of Medicare,798123,37063,898459
MSH,"HEALTHPLUS HP, LLC HEALTHPLUS M

# Check for newer version

https://www.mountsinai.org/about/insurance/msh/price-transparency

In [3]:
!wget https://www.mountsinai.org/files/MSHealth/Assets/HS/131624096_mount-sinai-hospital_standardcharges.zip -P /tmp

--2023-09-28 15:49:15--  https://www.mountsinai.org/files/MSHealth/Assets/HS/131624096_mount-sinai-hospital_standardcharges.zip
Resolving www.mountsinai.org (www.mountsinai.org)... 23.62.122.144
Connecting to www.mountsinai.org (www.mountsinai.org)|23.62.122.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74353186 (71M) [application/zip]
Saving to: ‘/tmp/131624096_mount-sinai-hospital_standardcharges.zip’

131624096_mount-sin 100%[===================>]  70.91M  3.93MB/s    in 16s     

2023-09-28 15:49:32 (4.32 MB/s) - ‘/tmp/131624096_mount-sinai-hospital_standardcharges.zip’ saved [74353186/74353186]



In [4]:
!unzip /tmp/131624096_mount-sinai-hospital_standardcharges.zip && mv 131624096_mount-sinai-hospital_standardcharges.txt /tmp/

Archive:  /tmp/131624096_mount-sinai-hospital_standardcharges.zip
  inflating: 131624096_mount-sinai-hospital_standardcharges.txt  


In [3]:
!head /tmp/131624096_mount-sinai-hospital_standardcharges.txt

facility_id|Code_type|Primary_Code|Revenue_Code|CPT_HCPCS|description|Service_Type|Gross_Charge|Contract_Name|Product_Name|Place_of_Service|Negotiated_Charge|Minimum_Negotiated_Charge|Maximum_Negotiated_Charge
MSH|APRDRG|0011|||Liver transplant &/or intestinal transplant|Packaged|n/a|Beacon|Beacon Health Strategies Medicaid|I|46952.41|44650.82|209805.79
MSH|APRDRG|0013|||Liver transplant &/or intestinal transplant|Packaged|n/a|Fidelis|Fidelis Essential Plan 3 and 4|I|60612.19|60612.19|215658.43
MSH|APRDRG|0014|||Liver transplant &/or intestinal transplant|Packaged|n/a|Beacon|Beacon Health Strategies Medicaid|I|133357.25|126820.13|425626.88
MSH|APRDRG|0021|||Heart &/or lung transplant|Packaged|n/a|Beacon|Beacon Health Strategies Medicaid|I|92179.33|87660.74|203336.77
MSH|APRDRG|0023|||Heart &/or lung transplant|Packaged|n/a|Affinity|Affinity Essential Plans 1-2|I|253224.44|109167.87|337805.86
MSH|APRDRG|0023|||Heart &/or lung transplant|Packaged|n/a|Empire Healthplus |Medicaid|I|112544.

# Prompt for Claude

````
```
%%sql
SELECT *
FROM read_csv('https://data.cityofnewyork.us/api/views/erm2-nwe9/rows.csv?accessType=DOWNLOAD',
    header=True,
    delim=',',
    quote='"',
    columns={'Unique Key': 'BIGINT',
    'Created Date': 'VARCHAR',
    'Closed Date': 'VARCHAR',
    'Agency': 'VARCHAR',
    'Agency Name': 'VARCHAR',
    'Complaint Type': 'VARCHAR',
    'Descriptor': 'VARCHAR',
    'Location Type': 'VARCHAR',
    'Incident Zip': 'VARCHAR',
    'Incident Address': 'VARCHAR',
    'Street Name': 'VARCHAR',
    'Cross Street 1': 'VARCHAR',
    'Cross Street 2': 'VARCHAR',
    'Intersection Street 1': 'VARCHAR',
    'Intersection Street 2': 'VARCHAR',
    'Address Type': 'VARCHAR',
    'City': 'VARCHAR',
    'Landmark': 'VARCHAR',
    'Facility Type': 'VARCHAR',
    'Status': 'VARCHAR',
    'Due Date': 'VARCHAR',
    'Resolution Description': 'VARCHAR',
    'Resolution Action Updated Date': 'VARCHAR',
    'Community Board': 'VARCHAR',
    'BBL': 'VARCHAR',
    'Borough': 'VARCHAR',
    'X Coordinate (State Plane)': 'VARCHAR',
    'Y Coordinate (State Plane)': 'VARCHAR',
    'Open Data Channel Type': 'VARCHAR',
    'Park Facility Name': 'VARCHAR',
    'Park Borough': 'VARCHAR',
    'Vehicle Type': 'VARCHAR',
    'Taxi Company Borough': 'VARCHAR',
    'Taxi Pick Up Location': 'VARCHAR',
    'Bridge Highway Name': 'VARCHAR',
    'Bridge Highway Direction': 'VARCHAR',
    'Road Ramp': 'VARCHAR',
    'Bridge Highway Segment': 'VARCHAR',
    'Latitude': 'DOUBLE',
    'Longitude': 'DOUBLE',
    'Location': 'VARCHAR'}) 
LIMIT 10;

Please take the above SQL query and rewrite for the file `131624096_mount-sinai-hospital_standardcharges.txt` with the following header:

```
facility_id|Code_type|Primary_Code|Revenue_Code|CPT_HCPCS|description|Service_Type|Gross_Charge|Contract_Name|Product_Name|Place_of_Service|Negotiated_Charge|Minimum_Negotiated_Charge|Maximum_Negotiated_Charge
MSH|APRDRG|0011|||Liver transplant &/or intestinal transplant|Packaged|n/a|Beacon|Beacon Health Strategies Medicaid|I|46952.41|44650.82|209805.79
MSH|APRDRG|0013|||Liver transplant &/or intestinal transplant|Packaged|n/a|Fidelis|Fidelis Essential Plan 3 and 4|I|60612.19|60612.19|215658.43
MSH|APRDRG|0014|||Liver transplant &/or intestinal transplant|Packaged|n/a|Beacon|Beacon Health Strategies Medicaid|I|133357.25|126820.13|425626.88
MSH|APRDRG|0021|||Heart &/or lung transplant|Packaged|n/a|Beacon|Beacon Health Strategies Medicaid|I|92179.33|87660.74|203336.77
MSH|APRDRG|0023|||Heart &/or lung transplant|Packaged|n/a|Affinity|Affinity Essential Plans 1-2|I|253224.44|109167.87|337805.86
MSH|APRDRG|0023|||Heart &/or lung transplant|Packaged|n/a|Empire Healthplus |Medicaid|I|112544.2|109167.87|337805.86
MSH|APRDRG|0023|||Heart &/or lung transplant|Packaged|n/a|Fidelis|Fidelis Exchange|I|202579.55|109167.87|337805.86
MSH|APRDRG|0033|||Bone marrow transplant|Packaged|n/a|MetroPlus|MetroPlus Essential Plan 3 & 4|I|88303.07|85653.98|256885.17
MSH|APRDRG|0041|||Tracheostomy w MV 96+ hours w extensive procedure or ECMO|Packaged|n/a|Affinity|Affinity Essential Plans 1-2|I|120801.21|52078.74|120801.21
```
```````

# Result from Claude

In [4]:
%%sql 
SELECT *
FROM read_csv('/tmp/131624096_mount-sinai-hospital_standardcharges.txt', 
  header=True, 
  delim='|',
  columns={
    'facility_id': 'VARCHAR',
    'Code_type': 'VARCHAR',
    'Primary_Code': 'VARCHAR', 
    'Revenue_Code': 'VARCHAR',
    'CPT_HCPCS': 'VARCHAR',
    'description': 'VARCHAR',
    'Service_Type': 'VARCHAR',
    'Gross_Charge': 'VARCHAR',
    'Contract_Name': 'VARCHAR',
    'Product_Name': 'VARCHAR',
    'Place_of_Service': 'VARCHAR', 
    'Negotiated_Charge': 'VARCHAR',
    'Minimum_Negotiated_Charge': 'VARCHAR',
    'Maximum_Negotiated_Charge': 'VARCHAR'})
LIMIT 10;

,facility_id,Code_type,Primary_Code,Revenue_Code,CPT_HCPCS,description,Service_Type,Gross_Charge,Contract_Name,Product_Name,Place_of_Service,Negotiated_Charge,Minimum_Negotiated_Charge,Maximum_Negotiated_Charge
0,MSH,APRDRG,0011,NaN,NaN,Liver transplant &/or intestinal transplant,Packaged,n/a,Beacon,Beacon Health Strategies Medicaid,I,46952.41,44650.82,209805.79
1,MSH,APRDRG,0013,NaN,NaN,Liver transplant &/or intestinal transplant,Packaged,n/a,Fidelis,Fidelis Essential Plan 3 and 4,I,60612.19,60612.19,215658.43
2,MSH,APRDRG,0014,NaN,NaN,Liver transplant &/or intestinal transplant,Packaged,n/a,Beacon,Beacon Health Strategies Medicaid,I,133357.25,126820.13,425626.88
3,MSH,APRDRG,0021,NaN,NaN,Heart &/or lung transplant,Packaged,n/a,Beacon,Beacon Health Strategies Medicaid,I,92179.33,87660.74,203336.77
4,MSH,APRDRG,0023,NaN,NaN,Heart &/or lung transplant,Packaged,n/a,Affinity,Affinity Essential Plans 1-2,I,253224.44,109167.87,337805.86
5,MSH,APRDRG,0023,NaN,NaN,Heart &/or lung transplant,Packaged,n/a,Empire Healthplus,Medicaid,I,112544.2,109167.87,337805.86
6,MSH,APRDRG,0023,NaN,NaN,Heart &/or lung transplant,Packaged,n/a,Fidelis,Fidelis Exchange,I,202579.55,109167.87,337805.86
7,MSH,APRDRG,0033,NaN,NaN,Bone marrow transplant,Packaged,n/a,MetroPlus,MetroPlus Essential Plan 3 & 4,I,88303.07,85653.98,256885.17
8,MSH,APRDRG,0041,NaN,NaN,Tracheostomy w MV 96+ hours w extensive proced...,Packaged,n/a,Affinity,Affinity Essential Plans 1-2,I,120801.21,52078.74,120801.21
9,MSH,APRDRG,0041,NaN,NaN,Tracheostomy w MV 96+ hours w extensive proced...,Packaged,n/a,Emblem,MSHS CAID,I,53689.42,52078.74,120801.21


# Edit the query from Claude
Using the parameters here: https://duckdb.org/docs/data/csv/overview.html#parameters
- [ ] add `nullstr`

In [6]:
%%sql 
SELECT *
FROM read_csv('/tmp/131624096_mount-sinai-hospital_standardcharges.txt', 
  header=True, 
  delim='|',
  nullstr='n/a',
  columns={
    'facility_id': 'VARCHAR',
    'Code_type': 'VARCHAR',
    'Primary_Code': 'VARCHAR', 
    'Revenue_Code': 'VARCHAR',
    'CPT_HCPCS': 'VARCHAR',
    'description': 'VARCHAR',
    'Service_Type': 'VARCHAR',
    'Gross_Charge': 'FLOAT',
    'Contract_Name': 'VARCHAR',
    'Product_Name': 'VARCHAR',
    'Place_of_Service': 'VARCHAR', 
    'Negotiated_Charge': 'FLOAT',
    'Minimum_Negotiated_Charge': 'FLOAT',
    'Maximum_Negotiated_Charge': 'FLOAT'})
LIMIT 10;

,facility_id,Code_type,Primary_Code,Revenue_Code,CPT_HCPCS,description,Service_Type,Gross_Charge,Contract_Name,Product_Name,Place_of_Service,Negotiated_Charge,Minimum_Negotiated_Charge,Maximum_Negotiated_Charge
0,MSH,APRDRG,0011,,,Liver transplant &/or intestinal transplant,Packaged,NaN,Beacon,Beacon Health Strategies Medicaid,I,46952.410156,44650.820312,209805.796875
1,MSH,APRDRG,0013,,,Liver transplant &/or intestinal transplant,Packaged,NaN,Fidelis,Fidelis Essential Plan 3 and 4,I,60612.191406,60612.191406,215658.437500
2,MSH,APRDRG,0014,,,Liver transplant &/or intestinal transplant,Packaged,NaN,Beacon,Beacon Health Strategies Medicaid,I,133357.250000,126820.132812,425626.875000
3,MSH,APRDRG,0021,,,Heart &/or lung transplant,Packaged,NaN,Beacon,Beacon Health Strategies Medicaid,I,92179.328125,87660.742188,203336.765625
4,MSH,APRDRG,0023,,,Heart &/or lung transplant,Packaged,NaN,Affinity,Affinity Essential Plans 1-2,I,253224.437500,109167.867188,337805.875000
5,MSH,APRDRG,0023,,,Heart &/or lung transplant,Packaged,NaN,Empire Healthplus,Medicaid,I,112544.203125,109167.867188,337805.875000
6,MSH,APRDRG,0023,,,Heart &/or lung transplant,Packaged,NaN,Fidelis,Fidelis Exchange,I,202579.546875,109167.867188,337805.875000
7,MSH,APRDRG,0033,,,Bone marrow transplant,Packaged,NaN,MetroPlus,MetroPlus Essential Plan 3 & 4,I,88303.070312,85653.976562,256885.171875
8,MSH,APRDRG,0041,,,Tracheostomy w MV 96+ hours w extensive proced...,Packaged,NaN,Affinity,Affinity Essential Plans 1-2,I,120801.210938,52078.738281,120801.210938
9,MSH,APRDRG,0041,,,Tracheostomy w MV 96+ hours w extensive proced...,Packaged,NaN,Emblem,MSHS CAID,I,53689.421875,52078.738281,120801.210938


# Copy to a Parquet file

In [7]:
%%sql
COPY (SELECT *
FROM read_csv('/tmp/131624096_mount-sinai-hospital_standardcharges.txt', 
  header=True, 
  delim='|',
  nullstr='n/a',
  columns={
    'facility_id': 'VARCHAR',
    'Code_type': 'VARCHAR',
    'Primary_Code': 'VARCHAR', 
    'Revenue_Code': 'VARCHAR',
    'CPT_HCPCS': 'VARCHAR',
    'description': 'VARCHAR',
    'Service_Type': 'VARCHAR',
    'Gross_Charge': 'FLOAT',
    'Contract_Name': 'VARCHAR',
    'Product_Name': 'VARCHAR',
    'Place_of_Service': 'VARCHAR', 
    'Negotiated_Charge': 'FLOAT',
    'Minimum_Negotiated_Charge': 'FLOAT',
    'Maximum_Negotiated_Charge': 'FLOAT'})
) TO '/Users/me/data/payless_health/EIN_131624096_mount-sinai.parquet' (COMPRESSION ZSTD);

,Success


In [12]:
%%sql
COPY (SELECT Primary_Code as billing_code, description, Product_Name AS health_insurance_name, Negotiated_Charge AS negotiated_rate
FROM read_csv('/tmp/131624096_mount-sinai-hospital_standardcharges.txt', 
  header=True, 
  delim='|',
  nullstr='n/a',
  columns={
    'facility_id': 'VARCHAR',
    'Code_type': 'VARCHAR',
    'Primary_Code': 'VARCHAR', 
    'Revenue_Code': 'VARCHAR',
    'CPT_HCPCS': 'VARCHAR',
    'description': 'VARCHAR',
    'Service_Type': 'VARCHAR',
    'Gross_Charge': 'FLOAT',
    'Contract_Name': 'VARCHAR',
    'Product_Name': 'VARCHAR',
    'Place_of_Service': 'VARCHAR', 
    'Negotiated_Charge': 'FLOAT',
    'Minimum_Negotiated_Charge': 'FLOAT',
    'Maximum_Negotiated_Charge': 'FLOAT'})
) TO '/Users/me/data/payless_health/EIN_131624096_mount-sinai.parquet' (COMPRESSION ZSTD);

,Success


In [13]:
%%sql 
SELECT * FROM '/Users/me/data/payless_health/EIN_131624096_mount-sinai.parquet'

,billing_code,description,health_insurance_name,negotiated_rate
0,0011,Liver transplant &/or intestinal transplant,Beacon Health Strategies Medicaid,46952.410156
1,0013,Liver transplant &/or intestinal transplant,Fidelis Essential Plan 3 and 4,60612.191406
2,0014,Liver transplant &/or intestinal transplant,Beacon Health Strategies Medicaid,133357.250000
3,0021,Heart &/or lung transplant,Beacon Health Strategies Medicaid,92179.328125
4,0023,Heart &/or lung transplant,Affinity Essential Plans 1-2,253224.437500
...,...,...,...,...
5995880,904,SKIN GRAFTS FOR INJURIES WITH CC/MCC,Cash Discount,43993.898438
5995881,908,OTHER O.R. PROCEDURES FOR INJURIES WITH CC,Cash Discount,25471.849609
5995882,919,COMPLICATIONS OF TREATMENT WITH MCC,Cash Discount,22215.570312
5995883,927,EXTENSIVE BURNS OR FULL THICKNESS BURNS WITH M...,Cash Discount,235113.906250


# Filter the Parquet file to include the maximum and minimum negotiated charges

In [8]:
%%sql 
SELECT DISTINCT
  Primary_Code,
  Code_type,
  description,
  MIN(Minimum_Negotiated_Charge) AS Min_Charge, 
  MAX(Maximum_Negotiated_Charge) AS Max_Charge
FROM '/Users/me/data/payless_health/EIN_131624096_mount-sinai.parquet'
GROUP BY Code_type, Primary_Code, description;

,Primary_Code,Code_type,description,Min_Charge,Max_Charge
0,A0065470,Charge_Code,ALLOGRAFT BLOCK 10MM PUROS,425.600006,972.799988
1,A0015317,Charge_Code,STENT DUAL URETERAL 6FR X 28C,99.790001,228.100006
2,A0014987,Charge_Code,BOOMERANG 36MM X 9MM TI,2561.570068,5855.009766
3,A0014593,Charge_Code,CROSSLINK X10 FOR 3.5 ROD 33M,385.000000,880.000000
4,A0011682,Charge_Code,HEAD 46MM HMRL MOD OFFST 19MM,853.299988,1950.400024
...,...,...,...,...,...
106733,45800,CPT_HCPCS,REPAIR RECT/BLADDER FISTULA,1290.060059,8637.070312
106734,96437,CPT_HCPCS,Chemotherapy - Other,808.000000,1771.310059
106735,A0006343,Charge_Code,CONE BODY 23MM 10MM,2768.679932,6328.399902
106736,93893,CPT_HCPCS,Tcd emboli detect w/inj,71.709999,1417.359985


Need to end up with 106k rows total; the unique primary code and code type. Now just need to return the corresponding product name.

In [9]:
%%sql 
CREATE TABLE standardcharges AS SELECT * FROM '/Users/me/data/payless_health/EIN_131624096_mount-sinai.parquet'

,Success


In [10]:
%%sql 
CREATE TABLE max_min_charges AS (
SELECT DISTINCT
  Primary_Code,
  Code_type,
  description,
  MIN(Minimum_Negotiated_Charge) AS Min_Charge, 
  MAX(Maximum_Negotiated_Charge) AS Max_Charge
FROM '/Users/me/data/payless_health/EIN_131624096_mount-sinai.parquet'
GROUP BY Code_type, Primary_Code, description
);

,Success


```
CREATE TABLE max_min_charges AS (
SELECT
  Primary_Code,
  Code_type,
  MIN(Minimum_Negotiated_Charge) AS Min_Charge, 
  MAX(Maximum_Negotiated_Charge) AS Max_Charge
FROM '131624096_mount-sinai-hospital_standardcharges.parquet'
GROUP BY Code_type, Primary_Code
);
```

create a CTE that uses this table and creates two new columns: `Product_Name_Minimum` (corresponding to the `Product_Name` associated with the minimum of the `Minimum_Negoitated_Charge` column called `Min_Charge`) and `Product_Name_Maximum` (corresponding to the `Product_Name` associated with the minimum of the `Maximum_Negoitated_Charge` column called `Max_Charge`) alongside the numerical value of the Min_Charge and Max_Charge columns



In [11]:
%%sql 
WITH max_min_charges AS (
  SELECT 
    Primary_Code,
    Code_type,
    description,
    MIN(Minimum_Negotiated_Charge) AS Min_Charge,
    MAX(Maximum_Negotiated_Charge) AS Max_Charge
  FROM '/Users/me/data/payless_health/EIN_131624096_mount-sinai.parquet'
  GROUP BY Code_type, Primary_Code, description
),

min_max_product_names AS (
  SELECT
    mmc.Primary_Code,
    mmc.Code_type,
    mmc.description, 
    mmc.Min_Charge,
    mmc.Max_Charge,
    FIRST_VALUE(sc.Product_Name) OVER (PARTITION BY mmc.Primary_Code, mmc.Code_type ORDER BY sc.Minimum_Negotiated_Charge ASC) AS Product_Name_Minimum,
    FIRST_VALUE(sc.Product_Name) OVER (PARTITION BY mmc.Primary_Code, mmc.Code_type ORDER BY sc.Maximum_Negotiated_Charge DESC) AS Product_Name_Maximum
  FROM max_min_charges mmc
  JOIN '/Users/me/data/payless_health/EIN_131624096_mount-sinai.parquet' sc
  ON mmc.Primary_Code = sc.Primary_Code AND mmc.Code_type = sc.Code_type
)
SELECT DISTINCT
  Primary_Code,
  Code_Type,
  description,
  Min_Charge,
  Max_Charge,
  Product_Name_Minimum,
  Product_Name_Maximum
FROM min_max_product_names;

,Primary_Code,Code_type,description,Min_Charge,Max_Charge,Product_Name_Minimum,Product_Name_Maximum
0,01443,CPT_HCPCS,Anesthesia Unit Rate,45.000000,45.000000,Amida Care Medicaid Managed Care,Amida Care Medicaid Managed Care
1,10385041,Charge_Code,AUTOMATED RBC COUNT,1.300000,31.139999,Amida Care Medicaid Managed Care,Commercial
2,1280294,Charge_Code,JC VIRUS QNT PCR,1.300000,443.309998,Beacon Health Strategies Medicare HMO,Beacon Health Strategies Exchange
3,1281267,Charge_Code,BENZODIAZEPINE SCREEN QT.S/P,1.300000,478.260010,United Oxford Government (Medicare),United Oxford Government (Medicare)
4,13133883,Charge_Code,INSERT ENDOVASC PROSTH TAA,1060.000000,2120.000000,HealthFirst Medicare,HealthFirst Medicare
...,...,...,...,...,...,...,...
106733,A0048071,Charge_Code,STENT ENDO 37MM CAROTID WALL,1536.500000,3512.000000,Pathway,MetroPlus Exchange
106734,A0039097,Charge_Code,NAIL IM 360MM 130D 10 GAMMA3,1246.310059,2848.699951,Individual,Bright Health Government - Medicare
106735,A0087313,Charge_Code,MESH SURG PRFX XL L2INXW1.5,132.649994,303.200012,HIP Commercial HMO,HIP Commercial HMO
106736,A0013181,Charge_Code,SCREW 5.0MM CANN CONI 80MM,140.669998,321.540009,MSHS CAID,MSHS CAID


In [36]:
%%sql 
COPY (
  WITH max_min_charges AS (
  SELECT 
    Primary_Code,
    Code_type,
    description,
    MIN(Minimum_Negotiated_Charge) AS Min_Charge,
    MAX(Maximum_Negotiated_Charge) AS Max_Charge
  FROM '131624096_mount-sinai-hospital_standardcharges.parquet'
  GROUP BY Code_type, Primary_Code, description
),

min_max_product_names AS (
  SELECT
    mmc.Primary_Code,
    mmc.Code_type,
    mmc.description, 
    mmc.Min_Charge,
    mmc.Max_Charge,
    FIRST_VALUE(sc.Product_Name) OVER (PARTITION BY mmc.Primary_Code, mmc.Code_type ORDER BY sc.Minimum_Negotiated_Charge ASC) AS Product_Name_Minimum,
    FIRST_VALUE(sc.Product_Name) OVER (PARTITION BY mmc.Primary_Code, mmc.Code_type ORDER BY sc.Maximum_Negotiated_Charge DESC) AS Product_Name_Maximum
  FROM max_min_charges mmc
  JOIN '131624096_mount-sinai-hospital_standardcharges.parquet' sc
  ON mmc.Primary_Code = sc.Primary_Code AND mmc.Code_type = sc.Code_type AND mmc.description = sc.description
)
SELECT DISTINCT
  Primary_Code,
  Code_Type,
  description,
  Min_Charge,
  Max_Charge,
  Product_Name_Minimum,
  Product_Name_Maximum
FROM min_max_product_names
) TO '/Users/me/projects/beta.payless.health/docs/public/data/131624096_mount-sinai-hospital_standardcharges-subset.parquet' (FORMAT 'parquet');


,Success


In [37]:
%%sql 
SELECT * FROM '/Users/me/projects/beta.payless.health/docs/public/data/131624096_mount-sinai-hospital_standardcharges-subset.parquet';

,Primary_Code,Code_type,description,Min_Charge,Max_Charge,Product_Name_Minimum,Product_Name_Maximum
0,0026U,CPT_HCPCS,Onc thyr dna&mrna 112 genes,2595.600098,6115.100098,Commercial,Commercial
1,00413,CPT_HCPCS,Anesthesia Unit Rate,45.000000,45.000000,Amida Care Medicaid Managed Care,Amida Care Medicaid Managed Care
2,00441,CPT_HCPCS,Anesthesia Unit Rate,45.000000,45.000000,Amida Care Medicaid Managed Care,Amida Care Medicaid Managed Care
3,01489,CPT_HCPCS,Anesthesia Unit Rate,45.000000,45.000000,Amida Care Medicaid Managed Care,Amida Care Medicaid Managed Care
4,01674,CPT_HCPCS,Anesthesia Unit Rate,45.000000,45.000000,Amida Care Medicaid Managed Care,Amida Care Medicaid Managed Care
...,...,...,...,...,...,...,...
106733,99996171,Charge_Code,HLTH BHV IVNTJ FAM W/O PT EA,25.180000,120.000000,Commercial,Commercial
106734,A0061370,Charge_Code,SCREW SELF TAP 4.0 X 12,51.799999,118.400002,Small Group,Primary Network
106735,A0010030,Charge_Code,SIDEPLATE OMEGA PLUS 145DEG,288.609985,659.679993,GHI Commercial PPO/EPO,GHI Commercial PPO/EPO
106736,A0022560,Charge_Code,CUP HUMERAL DELTA XTND +3MM,499.100006,1140.800049,MSHS CAID,Create Flex and Create IDN


In [39]:
import polars as pl
df = pl.read_parquet('/Users/me/projects/beta.payless.health/docs/public/data/131624096_mount-sinai-hospital_standardcharges-subset.parquet')

In [41]:
df['Primary_Code'].unique()

Primary_Code
str
"""X2101991"""
"""999D7260"""
"""152835"""
"""A0034647"""
"""A0006872"""
"""A0035145"""
"""A0017430"""
"""A0031397"""
"""A0065793"""


In [29]:
!ls -lh /Users/me/projects/beta.payless.health/docs/public/data/131624096_mount-sinai-hospital_standardcharges-subset.parquet

-rw-r--r--  1 me  staff   6.2M Aug  9 18:27 /Users/me/projects/beta.payless.health/docs/public/data/131624096_mount-sinai-hospital_standardcharges-subset.parquet


# Load parquet and visualize

In [2]:
import vegafusion as vf
import polars as pl
import altair as alt
from vega_datasets import data
alt.data_transformers.disable_max_rows()
alt.renderers.enable('html')

# Configure DuckDB connection
vf.runtime.set_connection("duckdb")

# Enable Mime Renderer
vf.enable(row_limit=100000000)

vegafusion.enable(mimetype='html', row_limit=100000000, embed_options=None)

In [3]:
# Load the chargemaster data into a Polars dataframe
charges = pl.read_parquet('131624096_mount-sinai-hospital_standardcharges.parquet')

In [4]:
print(charges.schema)

{'facility_id': Utf8, 'Code_type': Utf8, 'Primary_Code': Utf8, 'Revenue_Code': Utf8, 'CPT_HCPCS': Utf8, 'description': Utf8, 'Service_Type': Utf8, 'Gross_Charge': Float32, 'Contract_Name': Utf8, 'Product_Name': Utf8, 'Place_of_Service': Utf8, 'Negotiated_Charge': Float32, 'Minimum_Negotiated_Charge': Float32, 'Maximum_Negotiated_Charge': Float32}


In [18]:
# Create a bar chart
alt.Chart(charges).mark_bar().encode(
    x='Gross_Charge:Q',
    y='count()',
)

alt.Chart(...)

In [25]:
# Filter out cheap charges
filtered_charges = charges.filter(
    pl.col("Negotiated_Charge") > 10000
)



In [26]:
# Create a bar chart
alt.Chart(filtered_charges).mark_bar().encode(
    x='Gross_Charge:Q',
    y='count()',
)

alt.Chart(...)

# Select columns for export to smaller parquet file

In [4]:
%%sql 
SELECT * FROM '131624096_mount-sinai-hospital_standardcharges.parquet' LIMIT 10;

,facility_id,Code_type,Primary_Code,Revenue_Code,CPT_HCPCS,description,Service_Type,Gross_Charge,Contract_Name,Product_Name,Place_of_Service,Negotiated_Charge,Minimum_Negotiated_Charge,Maximum_Negotiated_Charge
0,MSH,APRDRG,0011,,,Liver transplant &/or intestinal transplant,Packaged,NaN,Beacon,Beacon Health Strategies Medicaid,I,46952.410156,44650.820312,209805.796875
1,MSH,APRDRG,0013,,,Liver transplant &/or intestinal transplant,Packaged,NaN,Fidelis,Fidelis Essential Plan 3 and 4,I,60612.191406,60612.191406,215658.437500
2,MSH,APRDRG,0014,,,Liver transplant &/or intestinal transplant,Packaged,NaN,Beacon,Beacon Health Strategies Medicaid,I,133357.250000,126820.132812,425626.875000
3,MSH,APRDRG,0021,,,Heart &/or lung transplant,Packaged,NaN,Beacon,Beacon Health Strategies Medicaid,I,92179.328125,87660.742188,203336.765625
4,MSH,APRDRG,0023,,,Heart &/or lung transplant,Packaged,NaN,Affinity,Affinity Essential Plans 1-2,I,253224.437500,109167.867188,337805.875000
5,MSH,APRDRG,0023,,,Heart &/or lung transplant,Packaged,NaN,Empire Healthplus,Medicaid,I,112544.203125,109167.867188,337805.875000
6,MSH,APRDRG,0023,,,Heart &/or lung transplant,Packaged,NaN,Fidelis,Fidelis Exchange,I,202579.546875,109167.867188,337805.875000
7,MSH,APRDRG,0033,,,Bone marrow transplant,Packaged,NaN,MetroPlus,MetroPlus Essential Plan 3 & 4,I,88303.070312,85653.976562,256885.171875
8,MSH,APRDRG,0041,,,Tracheostomy w MV 96+ hours w extensive proced...,Packaged,NaN,Affinity,Affinity Essential Plans 1-2,I,120801.210938,52078.738281,120801.210938
9,MSH,APRDRG,0041,,,Tracheostomy w MV 96+ hours w extensive proced...,Packaged,NaN,Emblem,MSHS CAID,I,53689.421875,52078.738281,120801.210938


In [18]:
%%sql
COPY (SELECT 
      Product_Name,
      Code_type,
      Primary_Code,
      description,
      Minimum_Negotiated_Charge,
      Maximum_Negotiated_Charge
      FROM '131624096_mount-sinai-hospital_standardcharges.parquet'
      -- LIMIT 1000 -- comment out if needed
) TO '131624096_mount-sinai-hospital_standardcharges-subset.parquet' (FORMAT 'parquet')

,Success


In [12]:
!ls -lh 131624096_mount-sinai-hospital_standardcharges-subset.parquet

-rw-r--r--@ 1 me  staff   3.3M Aug  9 16:46 131624096_mount-sinai-hospital_standardcharges-subset.parquet


In [13]:
!cp 131624096_mount-sinai-hospital_standardcharges-subset.parquet /Users/me/projects/beta.payless.health/docs/public/data

# Prompt for creating mosaic visualization
```
```
meta:
  title: Olympic Athletes
  description: >
    An interactive dashboard of athlete statistics.
    The input menus and searchbox filter the display and are automatically populated by backing data columns.
data:
  athletes: { file: data/athletes.parquet }
hconcat:
- vconcat:
  - hconcat:
    - input: menu
      label: Sport
      as: $query
      from: athletes
      column: sport
    - input: menu
      label: Sex
      as: $query
      from: athletes
      column: sex
    - input: search
      label: Name
      as: $query
      from: athletes
      column: name
      type: contains
  - vspace: 10
  - plot:
    - mark: dot
      data: { from: athletes, filterBy: $query }
      x: weight
      y: height
      fill: sex
      r: 2
      opacity: 0.1
    - mark: regressionY
      data: { from: athletes, filterBy: $query }
      x: weight
      y: height
      stroke: sex
    - select: intervalXY
      as: $query
      brush: { fillOpacity: 0, stroke: black }
    xyDomain: Fixed
    colorDomain: Fixed
    margins: { left: 35, top: 20, right: 1 }
    width: 570
    height: 350
  - vspace: 5
  - input: table
    from: athletes
    maxWidth: 570
    height: 250
    filterBy: $query
    columns: [name, nationality, sex, height, weight, sport]
    width: { name: 180, nationality: 100, sex: 50, height: 50, weight: 50, sport: 100 }
```

please rewrite using this file: `data/131624096_mount-sinai-hospital_standardcharges-subset.parquet` with the following columns:

```
      Code_type,
      Primary_Code,
      description,
      Minimum_Negotiated_Charge,
      Maximum_Negotiated_Charge
```
```

# Filter parquet file to a smaller size and without NaNs to debug mosaic